In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score, adjusted_mutual_info_score, confusion_matrix
import umap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

# Definir la clase KMeans personalizada
class Kmeans:
    def __init__(self, k=20, n_iter=500):
        self.k = k
        self.n_iter = n_iter
        self.labels = None
        self.centroids = None

    def fit(self , X):
        centroids = self.init_centroids(X)
        
        i = 0
        while(i < self.n_iter):
            labels = self.set_labels(X, centroids)
            new_centroids = self.update_centroids(X, labels)

            centroids = new_centroids
            i += 1
        self.centroids = centroids
        self.labels = labels

    def init_centroids(self , X):
        index_rand = np.random.choice(X.shape[0], self.k, replace=False)
        return X[index_rand]
    
    def set_labels(self , X, centroids):
        distances = np.sum((X[:, np.newaxis] - centroids)**2, axis=2)
        labels = np.argmin(distances, axis=1)
        return labels
    
    def update_centroids(self , X, labels):
        centroids = np.zeros((self.k, X.shape[1]))
        for i in range(self.k):
            cluster_points = X[labels == i]
            if len(cluster_points) > 0:
                centroids[i] = np.mean(cluster_points, axis=0)
        return centroids

# Cargar los datos de features
train_features_df = pd.read_csv('video_features_id.csv', header=None)
val_features_df = pd.read_csv('validation_features_id.csv', header=None)
test_features_df = pd.read_csv('test_features_id.csv', header=None)

# Cargar los datos originales para obtener las etiquetas
train_labels_df = pd.read_csv('train.csv')
val_labels_df = pd.read_csv('val.csv')
submission_template = pd.read_csv('sample_submission.csv')

# Asegurarnos de que las filas coincidan por ID
train_df = train_features_df.set_index(0).join(train_labels_df.set_index('youtube_id'), how='inner')
val_df = val_features_df.set_index(0).join(val_labels_df.set_index('youtube_id'), how='inner')
test_df = test_features_df.set_index(0)  # Solo características, sin etiquetas

# Separar los labels y los features en los datos de entrenamiento
train_labels = train_df['label']
train_features = train_df.drop(columns=['label'])

# Normalización de los datos
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features.dropna())

# Reducir la dimensionalidad usando UMAP
umap_reducer = umap.UMAP(n_components=50, random_state=42)
train_reduced_features = umap_reducer.fit_transform(train_features_scaled)

# Ejecutar KMeans personalizado
kmeans = Kmeans(k=train_labels.nunique())
kmeans.fit(train_reduced_features)

# Calcular y reportar métricas de clasificación
silhouette = silhouette_score(train_reduced_features, kmeans.labels)
print(f'Silhouette Score: {silhouette:.2f}')

# Graficar las métricas de clasificación
metrics = {'Silhouette Score': silhouette}
metric_names = list(metrics.keys())
metric_values = list(metrics.values())

plt.figure(figsize=(10, 6))
plt.bar(metric_names, metric_values, color=['blue', 'green', 'red'])
plt.xlabel('Métricas de Clasificación')
plt.ylabel('Valor')
plt.title('Métricas de Clasificación para KMeans con UMAP')
plt.ylim(0, 1)
plt.savefig('kmeans_metrics.png')
plt.show()

# Análisis y discusión de los resultados
print("\nAnálisis y Discusión de los Resultados:")
print("Silhouette Score mide qué tan similares son los puntos dentro del mismo clúster comparados con los puntos en diferentes clústeres.")

# Asignar clusters al conjunto de validación
val_features = val_df.drop(columns=['label'])
val_features_clean = val_features.dropna()
val_features_scaled = scaler.transform(val_features_clean)
val_reduced_features = umap_reducer.transform(val_features_scaled)
val_clusters = kmeans.set_labels(val_reduced_features, kmeans.centroids)

# Añadir los clusters predichos al DataFrame de validación
val_df_clean = val_df.loc[val_features_clean.index].copy()
val_df_clean['Predicted_Cluster'] = val_clusters

# Mapeo automático de clusters a etiquetas
cluster_to_label = {}
for cluster in set(val_clusters):
    cluster_labels = val_df_clean[val_df_clean['Predicted_Cluster'] == cluster]['label']
    most_common_label = cluster_labels.mode().iloc[0]  # Obtener la etiqueta más común en cada cluster
    cluster_to_label[cluster] = most_common_label

print("Mapeo de clusters a etiquetas:", cluster_to_label)

# Aplicar el mapeo
val_df_clean['Predicted_Label'] = val_df_clean['Predicted_Cluster'].map(cluster_to_label)

# Calcular la precisión de la validación
val_accuracy = sum(val_df_clean['label'] == val_df_clean['Predicted_Label']) / len(val_df_clean)
print(f'Precisión de la validación: {val_accuracy:.2f}')

# Crear y mostrar la matriz de confusión
conf_matrix = confusion_matrix(val_df_clean['label'], val_df_clean['Predicted_Label'])
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Matriz de Confusión para la Validación')
plt.savefig('confusion_matrix.png')
plt.show()

# Graficar los clusters
plt.figure(figsize=(10, 7))
sns.scatterplot(x=train_reduced_features[:, 0], y=train_reduced_features[:, 1], hue=kmeans.labels, palette='viridis')
plt.title('Clusters de KMeans en el conjunto de entrenamiento')
plt.xlabel('Componente UMAP 1')
plt.ylabel('Componente UMAP 2')
plt.legend(title='Cluster')
plt.savefig('kmeans_clusters.png')
plt.show()

# Preprocesar el conjunto de prueba de manera similar
test_features_clean = test_df.dropna()
test_features_scaled = scaler.transform(test_features_clean)
test_reduced_features = umap_reducer.transform(test_features_scaled)

# Predecir con KMeans personalizado
test_clusters = kmeans.set_labels(test_reduced_features, kmeans.centroids)

# Añadir los clusters predichos al DataFrame de prueba
test_df_clean = test_df.loc[test_features_clean.index].copy()
test_df_clean['Predicted_Cluster'] = test_clusters

# Aplicar el mapeo
test_df_clean['Predicted_Label'] = test_df_clean['Predicted_Cluster'].map(cluster_to_label)

# Guardar las predicciones del conjunto de prueba en el archivo CSV según la plantilla
submission = pd.DataFrame({
    'youtube_id': test_df_clean.index,
    'label': test_df_clean['Predicted_Label']
})
submission.to_csv('test_predictions.csv', index=False)

print("Predicciones del conjunto de prueba guardadas en 'test_predictions.csv'")
